### Metric test

In [1]:
from DataCompression.msc.Metric import Metric
import numpy as np
from DataCompression.src.decoder_MSE import CNN_AE
from DataCompression.msc.vae_agarap import AE
from DataCompression.src.buffer import Decoder_Buffer
import torch

In [2]:
# load the test dataset
path = f"C:/Users/jabad/Documents/Tuebingen/DataCompression/DataCompression/exp/test_dataset"
db = Decoder_Buffer([3, 210, 160], 128, 1000)
db.load(path)
images = db.images
latents = db.latents

Created a buffer for images of shape torch.Size([1000, 3, 210, 160]) and latents torch.Size([1000, 128])


In [3]:
# reformat the data to fit the decoder method
latent = torch.transpose(torch.unsqueeze(torch.unsqueeze(latents, -1), -1).float(), 2, 1)
x_train = images.float()

In [ ]:
# create the model

model = CNN_AE(in_dims=latent[0].shape, out_dims=x_train[0].shape)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())
epochs = 10

# train the model
epoch_losses = []
for epoch in range(epochs):
    total_loss = 0
    # reset the gradients to zero
    optimizer.zero_grad()

    # compute reconstructions
    outputs = model(latent)
    outputs = torch.squeeze(outputs, 0)

    # compute reconstruction loss
    loss = criterion(outputs, x_train)

    # compute gradients
    loss.backward()

    # perform parameter update
    optimizer.step()

    # add this images's loss to epoch losses (loss per image normalized)
    epoch_losses.append(loss.item() / len(images))

        # display the epoch training loss
    print("\n epoch : {}/{}, loss = {:.6f}".format(epoch + 1, epochs, epoch_losses[-1]))


 epoch : 1/10, loss = 5.592503

 epoch : 2/10, loss = 5.280712

 epoch : 3/10, loss = 4.949123

 epoch : 4/10, loss = 4.674606

 epoch : 5/10, loss = 4.451707


In [ ]:
# get the reconstructed images
outputs = model(latent)

In [ ]:
# run the metric
metric = Metric(torch.nn.MSELoss())
accuracy, bpd = metric.measure(x_train, outputs, latent)
print("MSE Loss:")
print(accuracy)
print("bits/image:")
print(bpd)

In [ ]:
# MSE Loss:
# 3619.858154296875
# bits/image:
# 0.576